In [2]:
import numpy
import json
import matplotlib.pyplot as plt
import sklearn
from pprint import pprint

Let's try loading in the dataset

In [9]:
train_set = json.load(open("train.json"))
m = len(train_set)

Maybe we could filter the ingredients by mapping each ingredient to the most-often used word in its name.

In [4]:
word_freq_dict = {}
for recipe in train_set:
    for ing in recipe['ingredients']:
        words = ing.strip().split(" ")
        for word in words:
            word
            if word in word_freq_dict:
                word_freq_dict[word] += 1
            else:
                word_freq_dict[word] = 1
pprint(word_freq_dict)

{u'': 67,
 u'&': 375,
 u"'N": 1,
 u"'n": 7,
 u'(': 20,
 u'(10': 5,
 u'(14': 2,
 u'(14.5': 3,
 u'(15': 3,
 u'(flour)': 3,
 u'(not': 5,
 u'(powder)': 4,
 u',': 1,
 u'-': 6,
 u'00': 1,
 u'1': 2,
 u'1%': 211,
 u'1/2': 2,
 u'100': 1,
 u'2': 2,
 u'2%': 85,
 u'25%': 3,
 u'3': 2,
 u'33%': 6,
 u'4': 11,
 u'40%': 12,
 u'43': 1,
 u'5': 4,
 u'7': 4,
 u'8': 2,
 u'95%': 4,
 u'96%': 1,
 u'A': 1,
 u'Accent': 6,
 u'Adobo': 1,
 u'Alaskan': 2,
 u'Ale': 3,
 u'Alexia': 1,
 u'Alfredo': 61,
 u'All': 9,
 u'All-Purpose': 3,
 u'Amarena': 1,
 u'Amaretti': 11,
 u'American': 35,
 u'Aminos': 4,
 u'Anaheim': 51,
 u'Andouille': 8,
 u'Angostura': 13,
 u'Apple': 9,
 u'Argo': 1,
 u'Arrabbiata': 1,
 u'Artichoke': 1,
 u'Artisan': 3,
 u'Asian': 4,
 u'Azteca': 1,
 u'BACARDI\xae': 2,
 u'BELL\xae': 3,
 u"BREAKSTONE'S": 1,
 u'Bacon': 5,
 u'Bags': 1,
 u'Baileys': 11,
 u'Baking': 13,
 u'Balance': 3,
 u'Balsamic': 1,
 u'Balsamico': 7,
 u'Barbecue': 1,
 u'Barilla': 4,
 u'Bars': 1,
 u'Bartlett': 2,
 u'Base': 3,
 u'Basil': 1,
 u'Bay

As you can see, the word "yellow" is used 3000 times, so this approach is not going to work.

Maybe we can pick out better words by using ingredients that are only one word long.Let's index these by cuisine.

In [24]:
names, onewords_by_cuis = [], []
for recipe in train_set:
    cuis = recipe['cuisine']
    if cuis not in names:
        i = len(names)
        names.append(cuis)
        onewords_by_cuis.append([])
    else:
        i=names.index(cuis)
    for item in recipe['ingredients']:
        if ' ' not in item and item not in onewords_by_cuis[i]:
            onewords_by_cuis[i].append(item)
print names[2]
pprint(onewords_by_cuis[2])

filipino
[u'eggs',
 u'pepper',
 u'salt',
 u'mayonaise',
 u'butter',
 u'sugar',
 u'cucumber',
 u'peppercorns',
 u'vinegar',
 u'fat',
 u'water',
 u'garlic',
 u'jicama',
 u'carrots',
 u'beansprouts',
 u'onions',
 u'strawberries',
 u'chives',
 u'orange',
 u'basil',
 u'filet',
 u'honey',
 u'steak',
 u'chicken',
 u'oil',
 u'lemongrass',
 u'margarine',
 u'ginger',
 u'sauce',
 u'potatoes',
 u'cilantro',
 u'tomatoes',
 u'corn',
 u'shallots',
 u'cabbage',
 u'pork',
 u'bananas',
 u'flour',
 u'ketchup',
 u'mango',
 u'shrimp',
 u'lemon',
 u'rolls',
 u'lime',
 u'fish',
 u'ampalaya',
 u'scallions',
 u'pig',
 u'rice',
 u'Sriracha',
 u'steamer',
 u'seasoning',
 u'tofu',
 u'sprite',
 u'cayenne',
 u'squid',
 u'tamarind',
 u'okra',
 u'horseradish',
 u'cantaloupe',
 u'papaya',
 u'tapioca',
 u'noodles',
 u'spinach',
 u'meat',
 u'raisins',
 u'cheese',
 u'ham',
 u'chorizo',
 u'leeks',
 u'annatto',
 u'beef',
 u'sirloin',
 u'beans',
 u'chili',
 u'jackfruit',
 u'taro',
 u'liver',
 u'crabmeat',
 u'peas',
 u'eggpl

There are misspellings in the ingredients.  Perhaps our code should not distinguish between "chili", "chilies", and "chillies".

Of course, we will need to strip all plurals (final "s" and "es").  Then, we can define a simple function that tells us if one word is a likely misspelling of another.

In [5]:
def eq(w_1, w_2):
    if (abs(len(w_1) - len(w_2)) >= 1 
        or min(len(w_1), len(w_2)) <= 4
        or w_1[0] != w_2[0]):
        return False
    w_2_x = "|"+w_2+"|"
    #generate hypotheses
    hyp_dir = [-1]*len(w_1)
    for ind in range(len(w_1)):
        poss = w_2_x[ind:ind+2]
        if w_1[ind] in poss:
            hyp_dir[ind] = poss.index(w_1[ind])
    left, right, center = hyp_dir.count(0), hyp_dir.count(2), hyp_dir.count(1)
    if abs(max([left, right, center])-len(w_1)) <= 1:
        return True
    else:
        return False

In [6]:
for i in word_freq_dict.keys():
    i = i.lower().rstrip("s").rstrip("e")
    for j in word_freq_dict.keys():
        j = j.lower().rstrip("s").rstrip("e")
        if i != j and eq(i, j):
            print i, j

bacon baton
kefalotyri kefalotiri
romano romana
canola canela
sliced spiced
cracker cracked
white® white,
blanco bianco
bream bread
bread bream
bread broad
chuck chunk
chuck chunk
rosemary rosemari
bacon baton
menth menta
meyer mayer
greas great
sprig sprit
canela canola
canela canola
marin mirin
beaten beater
linguisa linguica
water wafer
water wafer
water wafer
parsley parslei
parslei parsley
parslei parsley
sprit sprig
sprit split
sprit sprig
cracker cracked
melted malted
cracked cracker
cracked cracker
cracked cracker
split sprit
wafer water
wafer water
mirin marin
purpl purpo
meyer mayer
curry curly
breakstone’ breakstone'
shoot short
sprig sprit
stick stock
stick stock
crust crush
mayer meyer
mayer meyer
mayer maker
better beater
parsley parslei
curry curly
creamy creami
pozol posol
water wafer
water wafer
water wafer
shark sharp
shark sharp
shark shank
shark shank
creami creamy
creami creamy
creamy creami
purpo purpl
flank frank
flank frank
kidney kidnei
curly curry
curly curry


So, custard and mustard are not the same thing.  But muscavado and muscovado are.  How do we fix this?